# Step 13: Cosmos DB with Vector Search

This notebook demonstrates Cosmos DB integration with vector embeddings for semantic search.

**Note:** This example requires:
- Azure Cosmos DB for MongoDB
- Azure OpenAI embedding service
- Proper environment configuration

In [ ]:
import asyncio
from dataclasses import dataclass
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.data.vector import (
    VectorStoreField,
    vectorstoremodel,
    FieldTypes,
    DistanceFunction,
)
from semantic_kernel.connectors.azure_cosmos_db import (
    CosmosMongoStore,
    CosmosMongoCollection,
)
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
    AzureChatPromptExecutionSettings,
)

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="You appear to be connected to a CosmosDB cluster",
    category=UserWarning,
)

## Define Data Model with Vector Support

In [ ]:
@vectorstoremodel
@dataclass
class ChatHistoryModel:
    """Chat history model with vector embeddings for semantic search."""
    session_id: Annotated[str, VectorStoreField(field_type=FieldTypes.KEY)]
    user_id: Annotated[str, VectorStoreField(is_indexed=True)]
    messages: Annotated[list[dict[str, str]], VectorStoreField(is_indexed=True)]
    
    # Vector field for semantic search
    vector: Annotated[
        list[float] | None,
        VectorStoreField(
            field_type=FieldTypes.VECTOR,
            dimensions=3072,  # text-embedding-3-large dimensions
            distance_function=DistanceFunction.EUCLIDEAN_DISTANCE,
            embedding_generator=None,
        ),
    ] = None

## Custom ChatHistory with Vector Embeddings

In [ ]:
class ChatHistoryInCosmosDB(ChatHistory):
    """Extends ChatHistory with vector embedding support."""
    session_id: str
    user_id: str
    store: CosmosMongoStore
    collection: CosmosMongoCollection[str, ChatHistoryModel] | None = None

    async def create_collection(self, collection_name: str) -> None:
        self.collection = self.store.get_collection(
            ChatHistoryModel, collection_name=collection_name
        )

    async def store_messages(self) -> None:
        if self.collection:
            try:
                # Generate embedding from message content
                message_text = " ".join([msg.content for msg in self.messages if msg.content])
                vector_data = None
                if message_text:
                    embeddings = await embedding_service.generate_raw_embeddings([message_text])
                    vector_data = embeddings[0] if embeddings else None
                
                data = ChatHistoryModel(
                    session_id=self.session_id,
                    user_id=self.user_id,
                    messages=[msg.model_dump() for msg in self.messages],
                    vector=vector_data,
                )
                await self.collection.upsert(data)
            except Exception as e:
                print(f"Error storing messages: {e}")

    async def read_messages(self) -> None:
        self.messages.clear()
        if self.collection:
            record = await self.collection.get(self.session_id)
            if record:
                for message in record.messages:
                    self.messages.append(ChatMessageContent.model_validate(message))

## Setup Kernel and Services

In [ ]:
kernel = Kernel()
kernel.add_plugin(MathPlugin(), plugin_name="math")
kernel.add_plugin(TimePlugin(), plugin_name="time")

chat_service = AzureChatCompletion(service_id="default")
embedding_service = AzureTextEmbedding(service_id="embedding")

request_settings = AzureChatPromptExecutionSettings(service_id="default")
request_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(
    filters={"excluded_plugins": ["ChatBot"]}
)

kernel.add_service(chat_service)
kernel.add_service(embedding_service)

## Define Chat Loop

In [ ]:
async def chat(history: ChatHistoryInCosmosDB) -> bool:
    await history.read_messages()
    print(f"Loaded {len(history.messages)} messages from Cosmos DB")

    if len(history.messages) == 0:
        history.add_system_message(
            "You are MajidBot, a curious assistant that helps people figure out what they need."
        )
        history.add_user_message("Hi there, who are you?")
        history.add_assistant_message("I'm MajidBot! Ask me anything.")

    try:
        user_input = input("User:> ")
    except (KeyboardInterrupt, EOFError):
        print("\nExiting chat...")
        return False

    if user_input.lower().strip() == "exit":
        return False

    history.add_user_message(user_input)

    result = await chat_service.get_chat_message_content(
        history, request_settings, kernel=kernel
    )

    if result:
        print(f"MajidBot:> {result}")
        history.add_message(result)

    print(f"Storing {len(history.messages)} messages to Cosmos DB...")
    await history.store_messages()

    return True

## Run the Chat with Vector Search

Execute this to start the chat with semantic search capabilities.

In [ ]:
async def main():
    session_id = "session123"
    delete_when_done = False

    async with CosmosMongoStore() as store:
        history = ChatHistoryInCosmosDB(
            store=store, session_id=session_id, user_id="user"
        )
        await history.create_collection("chat_memory")

        print("Welcome to the MajidBot assistant! Type 'exit' to quit.")
        print("Try asking: What time is it? or What is 4 * 5?")

        chatting = True
        while chatting:
            chatting = await chat(history)

        if delete_when_done and history.collection:
            await history.collection.delete_collection()

await main()